In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [ ]:
## Read Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();

## define projection for use later
WCprojection = WC.first().projection();  
print('WorldCover projection:', WCprojection.getInfo());

In [ ]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/geo_ref.csv') 
print(boundary_georef.to_string())

# Extract albedo by city

In [ ]:
# set date range of interest for albedo calculation, image limit and albedo threshold

date_start = '2021-01-01'
date_end = '2022-01-01'
image_limit = 50 # max number of images to include, sorted from least to most cloudy

# define "low albedo" threshold
LowAlbedoMax = 0.20 # EnergyStar steep slope minimum initial value is 0.25. 3-year value is 0.15. https://www.energystar.gov/products/building_products/roof_products/key_product_criteria


In [ ]:
## Configure methods

# Read relevant Sentinel-2 data
S2 = ee.ImageCollection("COPERNICUS/S2_SR")
S2C = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

MAX_CLOUD_PROB=30
S2_ALBEDO_EQN='((B*Bw)+(G*Gw)+(R*Rw)+(NIR*NIRw)+(SWIR1*SWIR1w)+(SWIR2*SWIR2w))'
S2_VIZ = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3};


## METHODS

## get cloudmasked image collection 

def mask_and_count_clouds(s2wc,geom):
  s2wc=ee.Image(s2wc)
  geom=ee.Geometry(geom.geometry())
  is_cloud=ee.Image(s2wc.get('cloud_mask')).gt(MAX_CLOUD_PROB).rename('is_cloud')
  nb_cloudy_pixels=is_cloud.reduceRegion(
    reducer=ee.Reducer.sum().unweighted(), 
    geometry=geom, 
    scale=10, 
    maxPixels=1e9
   )
  return s2wc.updateMask(is_cloud.eq(0)).set('nb_cloudy_pixels',nb_cloudy_pixels.getNumber('is_cloud')).divide(10000)

def mask_clouds_and_rescale(im):
    clouds=ee.Image(im.get('cloud_mask')).select('probability')
    return im.updateMask(clouds.lt(MAX_CLOUD_PROB)).divide(10000)

def get_masked_s2_collection(roi,start,end):
  criteria=(ee.Filter.And(
            ee.Filter.date(start,end),
            ee.Filter.bounds(roi)
        ))
  s2=S2.filter(criteria)#.select('B2','B3','B4','B8','B11','B12')
  s2c=S2C.filter(criteria)
  s2_with_clouds=(ee.Join.saveFirst('cloud_mask').apply(**{
        'primary': ee.ImageCollection(s2),
        'secondary': ee.ImageCollection(s2c),
        'condition': ee.Filter.equals(**{'leftField':'system:index','rightField':'system:index'}) 
        }))
  def _mcc(im):
      return mask_and_count_clouds(im,roi) 
  #s2_with_clouds=ee.ImageCollection(s2_with_clouds).map(_mcc)
  #s2_with_clouds=s2_with_clouds.limit(image_limit,'nb_cloudy_pixels')
  s2_with_clouds=ee.ImageCollection(s2_with_clouds).map(mask_clouds_and_rescale)#.limit(image_limit,'CLOUDY_PIXEL_PERCENTAGE')
  return  ee.ImageCollection(s2_with_clouds)

# calculate albedo for images

# weights derived from 
# S. Bonafoni and A. Sekertekin, "Albedo Retrieval From Sentinel-2 by New Narrow-to-Broadband Conversion Coefficients," in IEEE Geoscience and Remote Sensing Letters, vol. 17, no. 9, pp. 1618-1622, Sept. 2020, doi: 10.1109/LGRS.2020.2967085.
def calc_s2_albedo(image):
  config={
    'Bw':0.2266,
    'Gw':0.1236,
    'Rw':0.1573,
    'NIRw':0.3417,
    'SWIR1w':0.1170,
    'SWIR2w':0.0338,
    'B':image.select('B2'),
    'G':image.select('B3'),
    'R':image.select('B4'),
    'NIR':image.select('B8'),
    'SWIR1':image.select('B11'),
    'SWIR2':image.select('B12')
  }
  return image.expression(S2_ALBEDO_EQN,config).double().rename('albedo')


In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## S2 MOSAIC AND ALBEDO
    dataset = get_masked_s2_collection(boundary_geo_ee,date_start,date_end)
    s2_albedo = dataset.map(calc_s2_albedo)
    mosaic=dataset.mean()
    albedoMean=s2_albedo.reduce(ee.Reducer.mean())
    albedoMean=albedoMean.multiply(100).round().toByte() # .toFloat() # # toByte() or toFloat() to reduce file size of export
    albedoMean=albedoMean.updateMask(albedoMean.gt(0)) # to mask 0/NoData values in toByte() format
    albedoMeanThres = albedoMean.updateMask(albedoMean.lt(LowAlbedoMax))

    # Download ee.Image of albedo as GeoTIFF
    geemap.ee_export_image_to_drive(
        albedoMean, 
        description = boundary_id + '-S2-albedo',
        folder='data', 
        scale=10, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-cities4forests' 

In [ ]:
out_dir = os.getcwd()

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-S2-albedo.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/albedo/sentinel-2/v_0/'+ boundary_id + '-S2-albedo-2021.tif',
                               ExtraArgs={'ACL':'public-read'})

# Visualize data

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=8)

In [ ]:
Map.addLayer(albedoMean),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)